In [1]:
# Import Tools

import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

In [2]:
tag_urls = {
    'Travel': 'https://medium.com/tag/travel/archive/{0}/{1:02d}/{2:02d}'
}

In [3]:
year = 2022
selected_days = [i for i in range(1, 48)] #Every day of the year

In [4]:
def convert_day(day):
    # if it is a leap year use month_days = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    m = 0
    d = 0
    while day > 0:
        d = day
        day -= month_days[m]
        m += 1
    return (m, d)

In [5]:
def get_claps(claps_str):
    if (claps_str is None) or (claps_str == '') or (claps_str.split is None):
        return 0
    split = claps_str.split('K')
    claps = float(split[0])
    claps = int(claps*1000) if len(split) == 2 else int(claps)
    return claps

In [6]:
articles_data = []
article_id = 0
n = len(selected_days)
for d in selected_days:
    month, day = convert_day(d)
    date = '{0}-{1:02d}-{2:02d}'.format(year, month, day) 
    print(f'{date}')
    for tag, url in tag_urls.items(): 
        response = requests.get(url.format(year, month, day), allow_redirects=True)
        if not response.url.startswith(url.format(year, month, day)):
            continue
        page = response.content
        soup = BeautifulSoup(page, 'html.parser')
        articles = soup.find_all(
            "div",
            class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
        
        for article in articles:
            
            title = article.find("h3", class_="graf--title")
            if title is None:
                continue
            title = title.contents[0]
            
            author = article.find_all("a")[0]['href'].split('?')[0].split('@')[1]
            author_url = article.find_all("a")[0]['href'].split('?')[0]
            
            subtitle = article.find("h4", class_="graf--subtitle")
            subtitle = subtitle.contents[0] if subtitle is not None else ''
            
            article_url = article.find_all("a")[3]['href'].split('?')[0]
            
            reading_time = article.find("span", class_="readingTime")
            reading_time = 0 if reading_time is None else int(reading_time['title'].split(' ')[0])
            
            responses = article.find_all("a")
            if len(responses) == 7:
                responses = responses[6].contents[0].split(' ')
                if len(responses) == 0:
                    responses = 0
                else:
                    responses = responses[0]
            else:
                responses = 0
                
            articles_data.append([article_url, title,
                         author, author_url,
                         subtitle, responses,
                         reading_time, tag, date])

2022-01-01
2022-01-02
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-15
2022-01-16
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16


In [8]:
# Transform article data into panda dataframe.
medium_df = pd.DataFrame(articles_data, columns=[
    'url', 'title', 'author', 'author_page',
    'subtitle', 'responses', 'reading_time',
    'tag', 'date'])

In [9]:
medium_df.shape

(4906, 9)

In [10]:
medium_df = medium_df.drop_duplicates(subset=['url', 'title'], keep='first')

In [11]:
medium_df.shape

(4906, 9)

In [12]:
medium_df

,url,title,author,author_page,subtitle,responses,reading_time,tag,date
0,https://medium.com/@anne.bonfert/i-got-stitche...,I Got Stitches in Paradise,anne.bonfert,https://medium.com/@anne.bonfert,Because I slipped on the tropical cliffs,0,9,Travel,2022-01-01
1,https://medium.com/jtv-insights/2021-year-in-r...,2021 Year in review,amy_burr,https://medium.com/@amy_burr,,0,3,Travel,2022-01-01
2,https://medium.com/@cassady-fendlay/the-nomad-...,The Nomad Year,cassady-fendlay,https://medium.com/@cassady-fendlay,,0,6,Travel,2022-01-01
3,https://medium.com/travel-experiences/the-one-...,The one before the Taj Mahal,shrinathv,https://medium.com/@shrinathv,,0,3,Travel,2022-01-01
4,https://medium.com/@stephencasmier/space-and-d...,"Hearing ""space and disjointed shit"" while flyi...",stephencasmier,https://medium.com/@stephencasmier,,0,7,Travel,2022-01-01
...,...,...,...,...,...,...,...,...,...
4901,https://medium.com/@hubpost/what-does-travel-m...,What does travel mean to you ?,hubpost,https://medium.com/@hubpost,,0,2,Travel,2022-02-16
4902,https://medium.com/@itsshirleygreen/planning-t...,[Planning travel around this season? Here are ...,itsshirleygreen,https://medium.com/@itsshirleygreen,,0,4,Travel,2022-02-16
4903,https://medium.com/@tratomaseliasgonzalezbenit...,Realiza viajes en automóvil,tratomaseliasgonzalezbenitez,https://medium.com/@tratomaseliasgonzalezbenitez,,0,0,Travel,2022-02-16
4904,https://medium.com/@alexjones8285-86083/ten-pl...,[Ten places not to miss when visiting Seychelles],alexjones8285-86083,https://medium.com/@alexjones8285-86083,,0,2,Travel,2022-02-16


In [13]:
medium_df.to_csv('medium-data-travel.csv', index=True)